In [ ]:
!pip install openai

In [ ]:
!pip install typing_extensions

In [ ]:
import openai
import pandas as pd
import os
import pickle
import csv
import numpy as np
import time
from openai import AsyncOpenAI

In [ ]:
client = AsyncOpenAI(api_key="OPENAI_API_KEY")

In [ ]:
prompt = """
which emotion does this segment of poem represent in single word"""

In [ ]:
messages = [{"role": "system", "content": prompt}]


In [ ]:
messages

[{'role': 'system',
  'content': '\ngenerate a prompt of this segment which visually represents this part of the segment '}]

In [ ]:
df = pd.read_csv("emnlp3000.csv")
posts = df["ctext"]
descriptions = []
for i in range(len(df)):
  message = posts[i]
  if message:
    if not isinstance(message, str):
      message = str(message)
    messages.append({"role": "user", "content": message})
    chat = await client.chat.completions.create(model="gpt-4o-mini", messages=messages)
  reply = chat.choices[0].message.content
  messages.pop()
  print(reply)
  descriptions.append(reply)


In [ ]:
!pip install rouge_score
!pip install evaluate
!pip install bert_score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=fa97f5e3aa2fdc26c5a31f10cf0c42cd243184e8f4aca1e113d87b0ae6b5491f
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 26.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 15.8 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's de

In [ ]:
from rouge_score import rouge_scorer
from nltk.translate.bleu_score import sentence_bleu
import pandas as pd
from evaluate import load

In [ ]:
# df = pd.read_csv("/content/train_poemsummation.csv")
# df1=pd.read_csv("/content/valid_poemsummation.csv")
# df = pd.concat([df, df1], axis=0)
# df= df.reset_index()

df = pd.read_csv("/content/mulimodal.csv")
df1 = pd.read_csv("/content/new_pixtral_outputs.csv")


scorer = rouge_scorer.RougeScorer(['rouge1','rouge2','rougeL'], use_stemmer=True)
rouge_1 = []
rouge_2 = []
rouge_l = []

bleu_1 = []
bleu_2 = []
bleu_3 = []
bleu_4 = []

bscore = []

bertscore = load("bertscore")
meteor = load('meteor')

meteor_score = []
for i in range(len(df)):

    src_sent = [str(df1['pix_outputs'][i])]
    tgt_sent = [str(df['Poem'][i])]
    results = meteor.compute(predictions=src_sent, references=tgt_sent)
    meteor_score.append(results['meteor'])

print("meteor score ",sum(meteor_score)/len(meteor_score))

for i in range(len(df)):

    src_sent = str(df1['pix_outputs'][i])
    tgt_sent = str(df['Poem'][i])

    scores = scorer.score(src_sent,tgt_sent)

    rouge_1.append(scores["rouge1"][2])
    rouge_2.append(scores["rouge2"][2])
    rouge_l.append(scores["rougeL"][2])

    bleu_1.append(sentence_bleu([tgt_sent],src_sent,weights=(1, 0, 0, 0)))
    bleu_2.append(sentence_bleu([tgt_sent],src_sent,weights=(0, 1, 0, 0)))
    bleu_3.append(sentence_bleu([tgt_sent],src_sent,weights=(0, 0, 1, 0)))
    bleu_4.append(sentence_bleu([tgt_sent],src_sent,weights=(0, 0, 0, 1)))




print("rouge 1",sum(rouge_1)/len(rouge_1))
print("rouge 2",sum(rouge_2)/len(rouge_2))
print("rouge L",sum(rouge_l)/len(rouge_l))


print("bleu 1",sum(bleu_1)/len(bleu_1))
print("bleu 2",sum(bleu_2)/len(bleu_2))
print("bleu 3",sum(bleu_3)/len(bleu_3))
print("bleu 4",sum(bleu_4)/len(bleu_4))



df = df.dropna(how="all")

for i in range(len(df)):

    predictions = [str(df1["pix_outputs"][i])]
    references = [str(df["Poem"][i])]

    k=bertscore.compute(predictions=predictions, references=references, lang="en")["f1"][0]
    bscore.append(k)

print("bert score",sum(bscore)/len(bscore))



[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


meteor score  0.3423989008029916


/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_

rouge 1 0.4097694462518612
rouge 2 0.31723648653859254
rouge L 0.390606389602211
bleu 1 0.515416342010099
bleu 2 0.37326081836724173
bleu 3 0.29928597435656246
bleu 4 0.2623524083277042


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


bert score 0.8605672657489777


In [ ]:
!python -m spacy download en_core_web_sm
!pip install spacy transformers sentence-transformers scikit-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 51.6 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
# Import statements for spaCy and preprocessing
import spacy


# Load spaCy model
nlp = spacy.load("en_core_web_sm")  # Make sure you have this model downloaded

# Import statements for sentiment analysis (using a BERT-based model)
from transformers import pipeline

from transformers import AutoModelForTokenClassification, AutoTokenizer

# Initialize the NER model
ner_model_name = "dbmdz/bert-large-cased-finetuned-conll03-english"
ner_model = pipeline("ner", model=AutoModelForTokenClassification.from_pretrained(ner_model_name),
                     tokenizer=AutoTokenizer.from_pretrained(ner_model_name))

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/998 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.33G [00:00<?, ?B/s]

Some weights of the model checkpoint at dbmdz/bert-large-cased-finetuned-conll03-english were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


tokenizer_config.json:   0%|          | 0.00/60.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

Device set to use cpu


In [ ]:
def preprocess_text(summary):
    doc = nlp(summary)  # Assuming spaCy's model is loaded as nlp
    tokens = [token.text for token in doc if not token.is_stop]
    pos_tags = [(token.text, token.pos_) for token in doc]
    return tokens, pos_tags

# Initialize the emotion classification pipeline
from transformers import pipeline
emotion_classifier = pipeline('text-classification', model='j-hartmann/emotion-english-distilroberta-base')

def extract_emotion_with_sentiment_analysis(summary):
    # Using a BERT-based sentiment analyzer
    emotion = emotion_classifier(summary)[0]
    current_emotion = emotion['label']
    return current_emotion

def extract_visual_elements_with_pos(summary):
    tokens, pos_tags = preprocess_text(summary)
    entities = ner_model(summary)
    visual_elements = [token.lower() for token, pos in pos_tags if pos in ['NOUN', 'PROPN']]  # Focus on nouns, proper nouns
    return visual_elements + [entity['word'].lower() for entity in entities]

from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Load pre-trained Sentence-BERT model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Define representative descriptions for each theme
themes = {
    "Positive/Negative Valence": "A theme reflecting a range of positive and negative emotions, such as love, joy, anger, or sorrow.",
    "Intensity of Emotion": "A theme exploring the intensity and depth of feelings or emotions.",
    "Love": "A theme centered around romantic love, familial love, or friendship.",
    "Anger": "A theme focused on expressions of anger, frustration, or conflict.",
    "Fear": "A theme involving fear, anxiety, or apprehension.",
    "Joy": "A theme celebrating happiness, joy, and the moments of delight.",
    "Sorrow": "A theme involving sadness, loss, grief, or sorrow.",
    "Existentialism": "A philosophical theme dealing with questions of existence, meaning, and the human condition.",
    "Human Condition": "A theme focused on the complexities, struggles, and experiences of being human.",
    "Freedom vs. Restriction": "A theme exploring the tension between freedom and restriction, liberation and control.",
    "Identity": "A theme centered around questions of self, identity, and personal growth.",
    "Social Justice": "A theme focusing on issues of fairness, equality, and social justice.",
    "Cultural Heritage": "A theme celebrating or questioning cultural traditions, practices, and heritage.",
    "Conflict and War": "A theme involving conflict, war, or struggle between opposing forces.",
    "Technology": "A theme exploring the impact of technology on society and individuals.",
    "Nature vs. Civilization": "A theme examining the contrast between natural environments and human-made societies.",
    "Ecological Awareness": "A theme emphasizing environmental awareness, conservation, or ecological concerns.",
    "Seasons and Cycles": "A theme reflecting on the cycles of nature, seasons, and their influence on life.",
    "Romantic Love": "A theme focused on romantic relationships and emotional connections.",
    "Family": "A theme centered around family relationships, bonds, and dynamics.",
    "Friendship": "A theme exploring the meaning and significance of friendship.",
    "Faith and Belief": "A theme involving spirituality, faith, and belief systems.",
    "Mortality and Afterlife": "A theme contemplating mortality, death, and the afterlife.",
    "Mental Struggles": "A theme dealing with psychological challenges, mental health, and inner conflicts.",
    "Dreams and Reality": "A theme blurring the lines between dreams and reality.",
    "Consciousness": "A theme exploring the nature of consciousness, self-awareness, and thought.",
    "Impermanence": "A theme reflecting on the transient nature of life, change, and time.",
    "Memory and Reflection": "A theme focused on memories, reflections, and the passage of time.",
    "Internal Conflict": "A theme involving personal struggles, dilemmas, and internal conflicts.",
    "External Conflict": "A theme focused on conflicts with external forces or situations.",
    "Resolution": "A theme involving the resolution of conflict, reconciliation, or peace.",
    "Uncertainty": "A theme exploring uncertainty, ambiguity, or the unknown.",
    "Curiosity and Exploration": "A theme centered around curiosity, exploration, and the pursuit of knowledge."
}

# Function to classify poem theme using cosine similarity
def classify_poem_theme(poem_summary):
    """
    Classify the theme of a poem summary using cosine similarity.

    Parameters:
    - poem_summary: A string containing the text of the poem summary.

    Returns:
    - The most relevant theme for the input summary.
    """
    # Compute the embedding for the poem summary
    poem_embedding = model.encode(poem_summary)

    # Compute embeddings for each theme description
    theme_embeddings = {theme: model.encode(description) for theme, description in themes.items()}

    # Calculate cosine similarity between the poem and each theme
    similarities = {theme: cosine_similarity([poem_embedding], [embedding])[0][0]
                    for theme, embedding in theme_embeddings.items()}

    # Select the theme with the highest similarity score
    best_theme = max(similarities, key=similarities.get)

    return best_theme

def extract_key_elements(summary):
    emotions = extract_emotion_with_sentiment_analysis(summary)
    visual_elements = extract_visual_elements_with_pos(summary)
    theme = classify_poem_theme(summary)
    print("Emotion:",emotions)
    print("Visual_elements",visual_elements)
    print("Theme",theme)
    return {
        "Primary Emotion": emotions,
        "Visual Elements": list(set(visual_elements)),
        "Theme": theme
    }



import pandas as pd

df = pd.read_csv("/content/summary.csv")
summaries=df["gpt_summary"]
emotion=[]
visual_Element=[]
theme=[]

for i in range(len(df)):
  summary = summaries[i][:2500]
  key_elements = extract_key_elements(summary)
  emotion.append(key_elements["Primary Emotion"])
  visual_Element.append(key_elements["Visual Elements"])
  theme.append(key_elements["Theme"])


df1=pd.DataFrame()
df1["our_summary"]=summaries[:len(emotion)]
df1["Emotion"]=emotion
df1["Visual Element"]=visual_Element
df1["Theme"]=theme
df1.to_csv("emelents_with_algo.csv")

config.json:   0%|          | 0.00/1.00k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/329M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/294 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Emotion: sadness
Visual_elements ['theme', 'poem', 'complexity', 'love', 'community', 'intertwining', 'pain', 'identity', 'phrase', 'love', 'supreme', 'mantra', 'transcendent', 'quality', 'love', 'mood', 'shifts', 'nature', 'love', 'source', 'suffering', 'joy', 'devices', 'repetition', 'phrase', 'love', 'supreme', 'theme', 'cadence', 'poem', 'imagery', 'landscape', 'realms', 'references', 'places', 'hamlet', 'north', 'carolina', 'elements', 'sweat', 'genitals', 'quality', 'experiences', 'tone', 'questions', 'identity', 'readers', 'poet', 'exploration', 'histories', 'references', 'elements', 'tenor', 'sax', 'love', 'pain', 'legacy', 'idea', 'art', 'struggle', 'loss', 'poem', 'love', 'experience', 'identity', 'african', 'american', 'hamlet', 'north', 'carolina', 'african', 'american']
Theme Positive/Negative Valence
Emotion: sadness
Visual_elements ['poem', 'theme', 'isolation', 'despair', 'parrot', 'entrapment', 'longing', 'freedom', 'mood', 'sadness', 'anger', 'sympathy', 'parrot', 'pl

KeyboardInterrupt: 

In [ ]:
%%capture
!pip install unsloth
# Also get the latest nightly Unsloth!
!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir --no-deps git+https://github.com/unslothai/unsloth.git

In [ ]:
from unsloth import FastVisionModel # FastLanguageModel for LLMs
import torch

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/Llama-3.2-11B-Vision-Instruct-bnb-4bit", # Llama 3.2 vision support
    "unsloth/Llama-3.2-11B-Vision-bnb-4bit",
    "unsloth/Llama-3.2-90B-Vision-Instruct-bnb-4bit", # Can fit in a 80GB card!
    "unsloth/Llama-3.2-90B-Vision-bnb-4bit",

    "unsloth/Pixtral-12B-2409-bnb-4bit",              # Pixtral fits in 16GB!
    "unsloth/Pixtral-12B-Base-2409-bnb-4bit",         # Pixtral base model

    "unsloth/Qwen2-VL-2B-Instruct-bnb-4bit",          # Qwen2 VL support
    "unsloth/Qwen2-VL-7B-Instruct-bnb-4bit",
    "unsloth/Qwen2-VL-72B-Instruct-bnb-4bit",

    "unsloth/llava-v1.6-mistral-7b-hf-bnb-4bit",      # Any Llava variant works!
    "unsloth/llava-1.5-7b-hf-bnb-4bit",
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastVisionModel.from_pretrained(
    "unsloth/Pixtral-12B-2409",
    load_in_4bit = True, # Use 4bit to reduce memory use. False for 16bit LoRA.
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for long context
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
Unsloth: OpenAI failed to import - ignoring for now.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.1.5: Fast Llava vision patching. Transformers: 4.47.1.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu121. CUDA: 7.5. CUDA Toolkit: 12.1. Triton: 3.1.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
df = pd.read_csv("/content/poems.csv")
df.dropna(inplace=True)
df.reset_index(drop=True, inplace=True)
df.to_csv("poems.csv")

In [ ]:
poems[0]

'A knight once brave, now old and worn,\nHis heart was heavy, spirit torn.\nHe rode alone through forests deep,\nWhere memories lay and secrets keep.\n\nHis sword once shone with battle’s light,\nNow rusted dull from lack of fight.\nHe sought a dragon, bold and wild,\nTo slay the beast as when a child.\n\nBut when he found its emerald eyes,\nHe saw no foe, but truth disguised.\nThe dragon bowed and spoke his name,\n"You seek a fight, but you\'re the same."\n\n"For years you chased a shadow’s woe,\nYet lost yourself so long ago."\nThe knight then dropped his weary blade,\nAnd let his war-torn past soon fade.'

In [ ]:
import os
import pandas as pd

poems =[]

with open("/content/short poems.txt",'r') as f:
  data=f.read()
  poems = data.split("\n\n\n\n\n")

segments = []
for poem in poems:
  segment = poem.split("\n\n")
  segments.append(segment)



df = pd.DataFrame()
df["poems"]=poems
df["segments"] = segments
df.to_csv("realpoems.csv")


In [ ]:
!cd /content/stanford-ner-2018-10-16

In [ ]:
import csv
import pandas as pd
import os

def append_to_csv(file_path, data):
    with open(file_path, 'a', newline='') as f:
        writer = csv.writer(f)
        writer.writerow(data)

def get_existing_row_count(file_path):
    if not os.path.exists(file_path):
        return 0
    with open(file_path, 'r') as f:
        reader = csv.reader(f)
        row_count = sum(1 for row in reader)
    return row_count-1

import nltk
nltk.download('punkt_tab')
df = pd.read_csv('/content/poems.csv')

# existing_row_count = get_existing_row_count('/content/modified_poems (8).csv')
column_names = ['poem', 'segments', 'ner_tags','emotions']
append_to_csv('modified_poems1.csv', column_names)

for index, row in df.iterrows():
    # if index < existing_row_count:
    #     continue

    poem = row['poems']
    segments, ner_tags, emotions = segment_poem_by_entities_and_emotion(poem)

    data_to_append = [
        row['poems'],
        segments,
        ner_tags,
        emotions
    ]

    append_to_csv('modified_poems1.csv', data_to_append)


[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


Sentence: From fairest creatures we desire increase , That thereby beauty ’ s rose might never die , But as the riper should by time decease , His tender heir might bear his memory : But thou , contracted to thine own bright eyes , Feed ’ st thy light ’ s flame with self-substantial fuel , Making a famine where abundance lies , Thyself thy foe , to thy sweet self too cruel : Thou that art now the world ’ s fresh ornament , And only herald to the gaudy spring , Within thine own bud buriest thy content , And tender churl mak ’ st waste in niggarding : Pity the world , or else this glutton be , To eat the world ’ s due , by the grave and thee .
NER Tags: [('From', 'O'), ('fairest', 'O'), ('creatures', 'O'), ('we', 'O'), ('desire', 'O'), ('increase', 'O'), (',', 'O'), ('That', 'O'), ('thereby', 'O'), ('beauty', 'O'), ('’', 'O'), ('s', 'O'), ('rose', 'O'), ('might', 'O'), ('never', 'O'), ('die', 'O'), (',', 'O'), ('But', 'O'), ('as', 'O'), ('the', 'O'), ('riper', 'O'), ('should', 'O'), ('by